<a href="https://colab.research.google.com/github/leahandofir/earthformer-inference-experiments/blob/main/earthformer_SEVIR_inference_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

SEVIR dataset: https://nbviewer.org/github/MIT-AI-Accelerator/eie-sevir/blob/master/examples/SEVIR_Tutorial.ipynb 

Install dependecies

In [ ]:
!curl 'https://raw.githubusercontent.com/leahandofir/earthformer-inference-experiments/main/prerequisites.sh' -o ./prerequisites.sh
!bash ./prerequisites.sh

In [11]:
# checking the aws cli installation
!aws --version

aws-cli/2.11.19 Python/3.11.3 Linux/5.10.147+ exe/x86_64.ubuntu.20 prompt/off


In [39]:
%env HOME=/content/earth-forecasting-transformer
%cd ~

env: HOME=/content/earth-forecasting-transformer
/content/earth-forecasting-transformer


In [40]:
# download the pre-trained weights
!curl 'https://deep-earth.s3.amazonaws.com/experiments/earthformer/pretrained_checkpoints/earthformer_sevir.pt' -o $HOME/tests/unittests/ earthformer_sevir.pt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0Warning: Failed to create the file 
  0 33.1M    0   488    0     0   1164      0  8:18:15 --:--:--  8:18:15  1164
curl: (23) Failed writing body (0 != 488)
curl: (6) Could not resolve host: earthformer_sevir.pt


In [ ]:
# SEVIR data files are h5 type files
!aws s3 ls --no-sign-request s3://sevir/data/vil --recursive --human-readable --region us-west-2
# download all 2019 data
!aws s3 cp --no-sign-request --recursive --region us-west-2 s3://sevir/data/vil/2019 ~/src/earthformer/datasets/sevir/data/vil
!aws s3 cp --no-sign-request --region us-west-2 s3://sevir/CATALOG.csv ~/src/earthformer/datasets/sevir/CATALOG.csv

2020-04-15 02:49:50    9.7 GiB data/vil/2017/SEVIR_VIL_RANDOMEVENTS_2017_0501_0831.h5
2020-04-15 02:49:50   12.8 GiB data/vil/2017/SEVIR_VIL_RANDOMEVENTS_2017_0901_1231.h5
2020-04-15 02:49:57    1.3 GiB data/vil/2017/SEVIR_VIL_STORMEVENTS_2017_0101_0630.h5
2020-04-15 02:50:00    4.6 GiB data/vil/2017/SEVIR_VIL_STORMEVENTS_2017_0701_1231.h5
2020-04-15 02:50:09   11.5 GiB data/vil/2018/SEVIR_VIL_RANDOMEVENTS_2018_0101_0430.h5
2020-04-15 02:52:12   15.9 GiB data/vil/2018/SEVIR_VIL_RANDOMEVENTS_2018_0501_0831.h5
2020-04-15 02:58:05   15.1 GiB data/vil/2018/SEVIR_VIL_RANDOMEVENTS_2018_0901_1231.h5
2020-04-15 03:06:49    5.3 GiB data/vil/2018/SEVIR_VIL_STORMEVENTS_2018_0101_0630.h5
2020-04-15 03:10:18    5.7 GiB data/vil/2018/SEVIR_VIL_STORMEVENTS_2018_0701_1231.h5
2020-04-15 18:28:01   15.2 GiB data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0101_0430.h5
2020-04-15 18:28:02   15.6 GiB data/vil/2019/SEVIR_VIL_RANDOMEVENTS_2019_0501_0831.h5
2020-04-15 18:28:02   15.1 GiB data/vil/2019/SEVIR_VIL_RAN

In [56]:
%cd ~/src
import sys, os
sys.path.append(os.path.join(os.getcwd(), '../tests/unittests'))
from test_pretrained_checkpoints import config_cuboid_transformer

from omegaconf import OmegaConf
import numpy as np
import torch
from earthformer.config import cfg
from earthformer.utils.layout import layout_to_in_out_slice
from earthformer.cuboid_transformer.cuboid_transformer import CuboidTransformerModel
from earthformer.datasets.sevir import SEVIRLightningDataModule

# load the pre-trained earthformer model
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
pretrained_cfg_path = os.path.join(cfg.root_dir, "scripts", "cuboid_transformer", "sevir", "earthformer_sevir_v1.yaml")
pretrained_cfg = OmegaConf.load(open(pretrained_cfg_path, "r"))
model = config_cuboid_transformer(
    cfg=pretrained_cfg,
    model_type="CuboidTransformerModel").to(device)
model.eval()
layout_cfg = pretrained_cfg.layout
in_slice, out_slice = layout_to_in_out_slice(layout=layout_cfg.layout,
                                              in_len=layout_cfg.in_len,
                                              out_len=layout_cfg.out_len)
# download data

# data_seq = batch['vil'].contiguous().to(device)
# x = data_seq[in_slice]
# y = data_seq[out_slice]
# y_hat = model(x)

/content/earth-forecasting-transformer/src


[slice(None, None, None),
 slice(None, 13, None),
 slice(None, None, None),
 slice(None, None, None),
 slice(None, None, None)]